In [ ]:
import boto3
import numpy as np
from pyspark.sql import SparkSession
from functools import partial

## Spark alone

In [ ]:
sc = SparkSession.builder.appName('Untitled').getOrCreate().sparkContext
print(sc)

In [ ]:
rdd = sc.parallelize([i for i in range(20)])

In [ ]:
def gen_data(n):
    return 20*(n.to_bytes(1,byteorder='little'))

In [ ]:
blocks = rdd.map(gen_data)

In [ ]:
blocks.collect()

## Boto3 alone

In [ ]:
s3 = boto3.client('s3')
mpu = s3.create_multipart_upload(Bucket='vmwishes-test',Key='junk3.dtb')
mpu['UploadId']

In [ ]:
head_part = s3.upload_part(Bucket='vmwishes-test',Key='junk3.dtb',
                           PartNumber=1,
                           UploadId=mpu['UploadId'],
                           Body=5400000*b'H')
parts = [{ 'PartNumber':1, 'ETag':head_part['ETag']}]

In [ ]:
for n in range(20):
    np = s3.upload_part(Bucket='vmwishes-test',Key='junk3.dtb',
                           PartNumber=2+n,
                           UploadId=mpu['UploadId'],
                           Body=5400000*(n.to_bytes(1,byteorder='little')))
    
    parts.append( {'PartNumber':2+n, 'ETag':np['ETag']} )
    print(f"Uploaded {n}: {np['ETag']}")
    

In [ ]:
parts

In [ ]:
result = s3.complete_multipart_upload(Bucket='vmwishes-test',
                                      Key='junk3.dtb',
                                      MultipartUpload={'Parts':parts},
                                      UploadId=mpu['UploadId'])

In [ ]:
s3.abort_multipart_upload(Bucket='vmwishes-test',
                                      Key='junk.dtb',
                                      UploadId=mpu['UploadId'])

In [ ]:
s3.list_multipart_uploads(Bucket='vmwishes-test')

## Boto3 + Spark

In [ ]:
sc = SparkSession.builder.appName('Untitled').getOrCreate().sparkContext
rdd = sc.parallelize([i for i in range(20)])

In [ ]:
s3 = boto3.client('s3')
mpu = s3.create_multipart_upload(Bucket='vmwishes-test',Key='junk4.dtb')
mpu['UploadId']

In [ ]:
def upload_func(uploadId, n):
    s3 = boto3.client('s3')
    try:
        part = s3.upload_part(Bucket='vmwishes-test',Key='junk4.dtb',
                              PartNumber=n+1,
                              UploadId=uploadId,
                              Body=5400000*(n.to_bytes(1,byteorder='little')))
    except Exception as e:
        return f'caught:{e}  n={n}  uploadId={uploadId}'
    
    return part['ETag']

In [ ]:
parts = rdd.map(partial(upload_func,mpu['UploadId'])).collect()
parts

In [ ]:
part_info = [ { 'PartNumber':i+1, 'ETag':etag } for i,etag in enumerate(parts) ]
part_info

In [ ]:
result = s3.complete_multipart_upload(Bucket='vmwishes-test',
                                      Key='junk4.dtb',
                                      MultipartUpload={'Parts':part_info},
                                      UploadId=mpu['UploadId'])

In [ ]:
s3.list_multipart_uploads(Bucket='vmwishes-test')

In [ ]:
s3.abort_multipart_upload(Bucket='vmwishes-test',
                                      Key='junk2.dtb',
                                      UploadId=mpu['UploadId'])